In [5]:
import pandas as pd
import os

In [6]:
def bayesian_adj_rating(df):
    min_votes = 30

    tmp_df = df.dropna(subset=['rating', 'num_votes'])

    tmp_df['w_rating'] = tmp_df['rating'] * tmp_df['num_votes'] / (tmp_df['num_votes'].sum())
    bayesian_weight = 4  # Bayesian 가중치 (조절 가능)
    tmp_df['b_rating'] = (
        (tmp_df['num_votes'] / (tmp_df['num_votes'] + min_votes)) * tmp_df['rating'] +
        (min_votes / (tmp_df['num_votes'] + min_votes)) * tmp_df['rating'].mean()
    )
    df = pd.merge(tmp_df[['wine_id', 'b_rating']], df, how='right', on='wine_id')

    return df

def get_item_data():
    data_path = "/home/dhkim/server_front/winery_server/data"
    data = pd.read_csv(os.path.join(data_path, "item_data.csv"))
    data = bayesian_adj_rating(data)
    # "grape"
    wine_column = ['wine_id','winetype','Red Fruit', 'Tropical', 'Tree Fruit', 'Oaky',\
        'Ageing', 'Black Fruit', 'Citrus', 'Dried Fruit', 'Earthy', 'Floral', \
        'Microbio','Spices', 'Vegetal', 'Light', 'Bold', 'Smooth', 'Tannic', 'Dry',\
        'Sweet', 'Soft', 'Acidic', 'Fizzy', 'Gentle','vintage','price',\
        'rating','b_rating', 'num_votes','country','region1','winery','name','wine_style','house',\
        'grape', 'pairing']

    # EX
    rename_rule = {
        'Red Fruit' : 'Red_Fruit',
        'Tree Fruit': 'Tree_Fruit',
        'Black Fruit':'Black_Fruit',
        'Dried Fruit':'Dried_Fruit',

    }

    df = data[wine_column]
    df = data.rename(columns=rename_rule)
    df.loc[df['vintage'].isna(), 'vintage'] = 0
    df[['grape','pairing']] = df[['grape','pairing']].astype(str)
    df['grape'] = df['grape'].copy().apply(lambda x: x.split() if x is not None else [])
    df['pairing'] = df['pairing'].copy().apply(lambda x: x.split() if x is not None else [])
    #df['winetype'].fillna(0, inplace=True)
    # for data in range(df.shape[0]):
    #     df['rating'][data] = float(df['rating'][data])
    return df

In [7]:
df = get_item_data()

<ipython-input-6-668ca154e54e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['w_rating'] = tmp_df['rating'] * tmp_df['num_votes'] / (tmp_df['num_votes'].sum())
<ipython-input-6-668ca154e54e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['b_rating'] = (


In [8]:
top_300_df = df.groupby('wine_style', group_keys=False).apply(lambda group: group.nlargest(300, 'b_rating'))
# Reset index if needed
top_300_df.reset_index(drop=True, inplace=True)
# Display the resulting DataFrame
top_300_df.loc[:,['wine_style','b_rating','wine_id']]

,wine_style,b_rating,wine_id
0,Alsace Gewürztraminer,4.406514,26005
1,Alsace Gewürztraminer,4.255158,25998
2,Alsace Gewürztraminer,4.232928,26001
3,Alsace Gewürztraminer,4.211794,26000
4,Alsace Gewürztraminer,4.206548,59638
...,...,...,...
23734,Washington State Red Blend,3.540134,41189
23735,Washington State Red Blend,3.525805,18899
23736,Washington State Red Blend,3.522408,55475
23737,Washington State Red Blend,3.514763,12199


In [9]:
unique_wine_styles = df['wine_style'].unique()
unique_wine_styles

array(['Bordeaux Red', "Italian Moscato d'Asti", 'Beaujolais Red', nan,
       'French Champagne', 'Loire Chenin Blanc', 'Californian Sparkling',
       'Washington State Red Blend', 'Bordeaux Pessac-Léognan',
       'Californian Chardonnay', 'Californian Pinot Noir',
       'German Riesling', 'Portuguese Dão Red', 'Argentinian Malbec',
       'Washington State Cabernet Sauvignon',
       'Californian Cabernet Sauvignon', 'Italian Amarone',
       'Bordeaux Saint-Estèphe', 'Provence Rosé', 'Southern Italy Red',
       'Italian Vino Nobile Di Montepulciano',
       'Austrian Grüner Veltliner', 'Bordeaux Margaux',
       'Southern Rhône Red', 'Northern Italy Red',
       'Languedoc-Roussillon Red', 'Burgundy Red', 'Northern Italy White',
       'Spanish Monastrell', 'Italian Barolo', 'Bordeaux Médoc',
       'Californian Red Blend', 'Bordeaux Saint-Émilion',
       'Chilean Cabernet Sauvignon', 'Chilean Chardonnay',
       'Upper Loire White', 'Southern Rhône White', 'Spanish Grenache',


In [15]:
def style_topK(df, style, K):
    if style != "Popular":
        filtered = df[df['wine_style'] == style]
    else:
        K = 100 
        filtered = df

    sorted_df = filtered.sort_values(by='b_rating', ascending=False)
    topK = sorted_df.head(K).loc[:, ['wine_id', 'name', 'rating','num_votes','b_rating']]
    return topK


In [16]:
mbti_styles = [
    "Popular",
    "Portuguese Port",
    "French Champagne",
    "Spanish Cava",
    "Italian Moscato d'Asti",
    "Californian Chardonnay",
    "Californian Sauvignon Blanc",
    "Bordeaux White",
    "Bourgogne Chablis",
    "German Riesling",
    "New Zealand Sauvignon Blanc",
    "Californian Pinot Noir",
    "Californian Cabernet Sauvignon",
    "Californian Merlot",
    "Italian Barolo",
    "Italian Barbaresco",
    "Italian Brunello di Montalcino",
    "Italian Chianti",
    "Beaujolais Red",
    "Bordeaux Red",
    "Bourgogne Red",
    "Argentinian Malbec",
    "Australian Shiraz",
    "Spanish Rioja Red"
]


In [20]:
styletopK = {}
for style in mbti_styles:
    styletopK[style] = list(style_topK(df, style, 500)['wine_id'])

In [21]:
import json
with open('/home/dhkim/server_front/winery_server/data/styletopK.json', 'w') as json_file:
    json.dump(styletopK, json_file)

In [14]:
 ['a1', 'b3', 'c2', 'd3', 'e2'],

(['a1', 'b3', 'c2', 'd3', 'e2'],)

In [4]:
df

NameError: name 'df' is not defined

In [34]:
df.columns

Index(['wine_id', 'b_rating', 'country', 'winery', 'winetype', 'name',
       'vintage', 'house', 'price', 'rating', 'num_votes', 'star5', 'star4',
       'star3', 'star2', 'star1', 'pairing', 'wine_style', 'Red_Fruit',
       'Tropical', 'Tree_Fruit', 'Oaky', 'Ageing', 'Black_Fruit', 'Citrus',
       'Dried_Fruit', 'Earthy', 'Floral', 'Microbio', 'Spices', 'Vegetal',
       'region2', 'region1', 'region3', 'region4', 'region5',
       'alcohol content', 'allergens', 'grape', 'wine style', 'grape_detail',
       'Light', 'Bold', 'Smooth', 'Tannic', 'Dry', 'Sweet', 'Soft', 'Acidic',
       'Fizzy', 'Gentle'],
      dtype='object')